Import pandas library, aliased as 'pd'

In [4]:
import pandas as pd

In [5]:
houses_columnwise = {
    "price_approx_usd": [114910.23, 423912.12, 23839.23,325912.32,83235.12],
    "surface_covered_in_m2":[128.0,239.2,83.2, 934.2,122.1],
    "rooms": [4.0,3.0,2.0,3.0,3.0]
}

In [6]:
houses_columnwise

{'price_approx_usd': [114910.23, 423912.12, 23839.23, 325912.32, 83235.12],
 'surface_covered_in_m2': [128.0, 239.2, 83.2, 934.2, 122.1],
 'rooms': [4.0, 3.0, 2.0, 3.0, 3.0]}

In [7]:
df_houses = pd.DataFrame(houses_columnwise)
df_houses

,price_approx_usd,surface_covered_in_m2,rooms
0,114910.23,128.0,4.0
1,423912.12,239.2,3.0
2,23839.23,83.2,2.0
3,325912.32,934.2,3.0
4,83235.12,122.1,3.0


In [8]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.utils.validation import check_is_fitted

In [9]:
df = pd.read_csv("mexico/mexico-city-real-estate-1.csv")
df

,Unnamed: 0,operation,property_type,place_with_parent_names,lat-lon,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url
0,1,sell,apartment,|Miguel Hidalgo|Distrito Federal|México|,"23.634501,-102.552788",5500000.0,MXN,5450245.50,289775.66,NaN,54.0,NaN,101851.851900,NaN,NaN,NaN,http://miguel-hidalgo-df.properati.com.mx/o3zb...
1,2,sell,house,|Iztapalapa|Distrito Federal|México|,"19.31033,-99.068557",1512000.0,MXN,1498321.97,79661.96,NaN,80.0,NaN,18900.000000,NaN,NaN,NaN,http://iztapalapa.properati.com.mx/q7t0_venta_...
2,3,sell,apartment,|Tlalpan|Distrito Federal|México|,"19.279771,-99.234597",926667.0,MXN,918284.00,48822.82,NaN,100.0,NaN,9266.670000,NaN,NaN,NaN,http://tlalpan.properati.com.mx/qbi4_venta_dep...
3,4,sell,apartment,|Miguel Hidalgo|Distrito Federal|México|,"23.634501,-102.552788",6410000.0,MXN,6352013.39,337720.36,NaN,135.0,NaN,47481.481480,NaN,NaN,NaN,http://miguel-hidalgo-df.properati.com.mx/opeq...
4,5,sell,apartment,|Benito Juárez|Quintana Roo|México|,"21.1902642,-86.8198375",875000.0,USD,16457437.50,875000.00,0.0,263.0,NaN,3326.996198,NaN,NaN,NaN,http://cancun.properati.com.mx/hg4t_venta_depa...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4623,4624,sell,apartment,|Benito Juárez|Quintana Roo|México|,"20.797201,-86.967773",200000.0,USD,3761700.00,200000.00,NaN,240.0,NaN,833.333333,NaN,1.0,NaN,http://cancun.properati.com.mx/fmik_venta_depa...
4624,4625,sell,apartment,|Tlalpan|Distrito Federal|México|,"19.303758,-99.223733",10000000.0,MXN,10144274.75,539345.23,200.0,NaN,2696.726150,NaN,NaN,NaN,NaN,http://lomas-del-pedregal.properati.com.mx/pym...
4625,4626,sell,house,|Solidaridad|Quintana Roo|México|,"20.613375,-87.089897",480000.0,USD,9028080.00,480000.00,451.0,245.0,1064.301552,1959.183673,2.0,NaN,NaN,http://playa-del-carmen.properati.com.mx/bvwt_...
4626,4627,sell,apartment,|Cuauhtémoc|Distrito Federal|México|,"23.634501,-102.552788",2453800.0,MXN,2431602.18,129282.09,NaN,67.0,NaN,36623.880600,NaN,NaN,NaN,http://cuauhtemoc.properati.com.mx/p2op_venta_...


In [83]:
def wrangle(filepath):
    df = pd.read_csv(filepath)
    
    rem_parNames = df['place_with_parent_names'].str.contains('Distrito Federal')
    rem_apt = df['property_type'] == 'apartment'
    rem_price = df['price_aprox_usd'] < 400_000
    rem_currency = df['currency'] == 'MXN'
    df = df[rem_parNames & rem_apt & rem_price & rem_currency]
    df[['lat', 'lon']] = df['lat-lon'].str.split(',', expand= True).astype(float)
    df.drop(columns= 'lat-lon', inplace=True)
    
    return df


In [84]:
frame1 = wrangle('mexico/mexico-city-real-estate-1.csv')
frame1.head()

,Unnamed: 0,operation,property_type,place_with_parent_names,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,lat,lon
0,1,sell,apartment,|Miguel Hidalgo|Distrito Federal|México|,5500000.0,MXN,5450245.50,289775.66,NaN,54.0,NaN,101851.85190,NaN,NaN,NaN,http://miguel-hidalgo-df.properati.com.mx/o3zb...,23.634501,-102.552788
2,3,sell,apartment,|Tlalpan|Distrito Federal|México|,926667.0,MXN,918284.00,48822.82,NaN,100.0,NaN,9266.67000,NaN,NaN,NaN,http://tlalpan.properati.com.mx/qbi4_venta_dep...,19.279771,-99.234597
3,4,sell,apartment,|Miguel Hidalgo|Distrito Federal|México|,6410000.0,MXN,6352013.39,337720.36,NaN,135.0,NaN,47481.48148,NaN,NaN,NaN,http://miguel-hidalgo-df.properati.com.mx/opeq...,23.634501,-102.552788
5,6,sell,apartment,|Miguel Hidalgo|Distrito Federal|México|,4416000.0,MXN,4376051.62,232663.51,NaN,87.0,NaN,50758.62069,NaN,NaN,NaN,http://anzures.properati.com.mx/opct_venta_dep...,19.432657,-99.177444
6,7,sell,apartment,|Benito Juárez|Distrito Federal|México|,3150000.0,MXN,3122244.47,166001.78,NaN,100.0,NaN,31500.00000,NaN,NaN,NaN,http://benito-juarez.properati.com.mx/r3a9_ven...,19.367025,-99.170349


In [85]:
frame1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2424 entries, 0 to 4627
Data columns (total 18 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Unnamed: 0                  2424 non-null   int64  
 1   operation                   2424 non-null   object 
 2   property_type               2424 non-null   object 
 3   place_with_parent_names     2424 non-null   object 
 4   price                       2424 non-null   float64
 5   currency                    2424 non-null   object 
 6   price_aprox_local_currency  2424 non-null   float64
 7   price_aprox_usd             2424 non-null   float64
 8   surface_total_in_m2         481 non-null    float64
 9   surface_covered_in_m2       2308 non-null   float64
 10  price_usd_per_m2            304 non-null    float64
 11  price_per_m2                2282 non-null   float64
 12  floor                       0 non-null      float64
 13  rooms                       61 non-nul

In [86]:
frame1.shape

(2424, 18)

In [87]:
len(df.columns)

19

In [88]:
df.columns

Index(['Unnamed: 0', 'operation', 'property_type', 'place_with_parent_names',
       'lat-lon', 'price', 'currency', 'price_aprox_local_currency',
       'price_aprox_usd', 'surface_total_in_m2', 'surface_covered_in_m2',
       'price_usd_per_m2', 'price_per_m2', 'floor', 'rooms', 'expenses',
       'properati_url', 'lat', 'lon'],
      dtype='object')

In [89]:
frame1['currency'].value_counts()

currency
MXN    2424
Name: count, dtype: int64

In [90]:
frame1['property_type'].value_counts()

property_type
apartment    2424
Name: count, dtype: int64

In [91]:
frame1['place_with_parent_names'].str.contains("Distrito Federal").value_counts()

place_with_parent_names
True    2424
Name: count, dtype: int64

In [93]:
frame2 = wrangle('mexico/mexico-city-real-estate-2.csv')
frame2

,Unnamed: 0,operation,property_type,place_with_parent_names,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,lat,lon
0,1,sell,apartment,|Gustavo A. Madero|Distrito Federal|México|,1200000.0,MXN,1189144.46,63223.78,88.0,88.0,718.452046,13636.363640,NaN,NaN,NaN,http://gustavo-a-madero.properati.com.mx/r644_...,19.516777,-99.160149
1,2,sell,apartment,|Gustavo A. Madero|Distrito Federal|México|,480000.0,MXN,475657.74,25289.51,NaN,48.0,NaN,10000.000000,NaN,NaN,NaN,http://vallejo.properati.com.mx/mz4c_venta_dep...,19.466724,-99.131614
2,3,sell,apartment,|Álvaro Obregón|Distrito Federal|México|,4559261.0,MXN,4518016.68,240211.43,NaN,126.0,NaN,36184.611110,NaN,NaN,NaN,http://alvaro-obregon.properati.com.mx/jh8j_ve...,19.332280,-99.243934
4,5,sell,apartment,|Benito Juárez|Distrito Federal|México|,1730000.0,MXN,1714349.82,91147.61,NaN,100.0,NaN,17300.000000,NaN,NaN,NaN,http://benito-juarez.properati.com.mx/q5my_ven...,19.393650,-99.147690
7,8,sell,apartment,|Cuauhtémoc|Distrito Federal|México|,1792666.0,MXN,1776870.21,94471.66,NaN,1200.0,NaN,1493.888333,NaN,NaN,NaN,http://cuauhtemoc.properati.com.mx/rccj_venta_...,19.420356,-99.143597
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4617,4618,sell,apartment,|Cuauhtémoc|Distrito Federal|México|,1243000.0,MXN,1231755.31,65489.29,NaN,150.0,NaN,8286.666667,NaN,NaN,NaN,http://cuauhtemoc.properati.com.mx/a1r7_venta_...,19.440177,-99.127123
4620,4621,sell,apartment,|Tlalpan|Distrito Federal|México|,1113000.0,MXN,1102931.38,58640.05,NaN,80.0,NaN,13912.500000,NaN,NaN,NaN,http://villa-lazaro-cardenas.properati.com.mx/...,19.295734,-99.146996
4623,4624,sell,apartment,|Cuauhtémoc|Distrito Federal|México|,739000.0,MXN,732314.77,38935.31,NaN,76.0,NaN,9723.684211,NaN,NaN,NaN,http://cuauhtemoc.properati.com.mx/izni_venta_...,23.634501,-102.552788
4624,4625,sell,apartment,|Cuauhtémoc|Distrito Federal|México|,3845484.0,MXN,3810696.70,202605.03,NaN,90.0,NaN,42727.600000,NaN,NaN,NaN,http://cuauhtemoc.properati.com.mx/q6qb_venta_...,19.409369,-99.171906


In [75]:
frame2.shape

(4628, 17)

In [76]:
frame1.shape

(2424, 19)

In [77]:
df = pd.concat([frame1,frame2], ignore_index=True)
df

,Unnamed: 0,operation,property_type,place_with_parent_names,lat-lon,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,lat,lon
0,1,sell,apartment,|Miguel Hidalgo|Distrito Federal|México|,"23.634501,-102.552788",5500000.0,MXN,5450245.50,289775.66,NaN,54.0,NaN,101851.851900,NaN,NaN,NaN,http://miguel-hidalgo-df.properati.com.mx/o3zb...,23.634501,-102.552788
1,3,sell,apartment,|Tlalpan|Distrito Federal|México|,"19.279771,-99.234597",926667.0,MXN,918284.00,48822.82,NaN,100.0,NaN,9266.670000,NaN,NaN,NaN,http://tlalpan.properati.com.mx/qbi4_venta_dep...,19.279771,-99.234597
2,4,sell,apartment,|Miguel Hidalgo|Distrito Federal|México|,"23.634501,-102.552788",6410000.0,MXN,6352013.39,337720.36,NaN,135.0,NaN,47481.481480,NaN,NaN,NaN,http://miguel-hidalgo-df.properati.com.mx/opeq...,23.634501,-102.552788
3,6,sell,apartment,|Miguel Hidalgo|Distrito Federal|México|,"19.432657,-99.177444",4416000.0,MXN,4376051.62,232663.51,NaN,87.0,NaN,50758.620690,NaN,NaN,NaN,http://anzures.properati.com.mx/opct_venta_dep...,19.432657,-99.177444
4,7,sell,apartment,|Benito Juárez|Distrito Federal|México|,"19.367025,-99.170349",3150000.0,MXN,3122244.47,166001.78,NaN,100.0,NaN,31500.000000,NaN,NaN,NaN,http://benito-juarez.properati.com.mx/r3a9_ven...,19.367025,-99.170349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7047,4624,sell,apartment,|Cuauhtémoc|Distrito Federal|México|,"23.634501,-102.552788",739000.0,MXN,732314.77,38935.31,NaN,76.0,NaN,9723.684211,NaN,NaN,NaN,http://cuauhtemoc.properati.com.mx/izni_venta_...,NaN,NaN
7048,4625,sell,apartment,|Cuauhtémoc|Distrito Federal|México|,"19.409369,-99.171906",3845484.0,MXN,3810696.70,202605.03,NaN,90.0,NaN,42727.600000,NaN,NaN,NaN,http://cuauhtemoc.properati.com.mx/q6qb_venta_...,NaN,NaN
7049,4626,sell,apartment,|Coyoacán|Distrito Federal|México|,NaN,6500000.0,MXN,6441199.16,342462.14,NaN,141.0,NaN,46099.290780,NaN,NaN,NaN,http://la-otra-banda-coyoacan.properati.com.mx...,NaN,NaN
7050,4627,sell,house,|Benito Juárez|Quintana Roo|México|,"21.139086,-86.84964",777106.0,MXN,770076.03,40942.98,NaN,82.0,NaN,9476.902439,NaN,NaN,NaN,http://benito-juarez-quintana-roo.properati.co...,NaN,NaN


In [78]:
df.shape

(7052, 19)